Market Basket Analysis

In [12]:
!pip3 install efficient-apriori
!pip3 install pymongo
!pip3 install pandas
!pip3 install matplotlib
!pip3 install seaborn
!pip3 install boto3
!pip3 install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from efficient_apriori import apriori
from IPython.display import display
import boto3
import os
from dotenv import load_dotenv

In [34]:
load_dotenv()
access_key_id = os.getenv("ACCESS_KEY_ID")
secret_access_key = os.getenv("SECRET_ACCESS_KEY")

bucket_name = 'is459-t3-job-transformed-data'
google_object_key = 'google_transformed/google.csv'
nodeflair_object_key = 'nodeflair_transformed/jobs.csv'

s3 = boto3.client('s3', aws_access_key_id=access_key_id, aws_secret_access_key=secret_access_key)


jobs_df = pd.read_csv(s3.get_object(Bucket=bucket_name, Key=nodeflair_object_key)['Body'])
google_df = pd.read_csv(s3.get_object(Bucket=bucket_name, Key=google_object_key)['Body'])

In [35]:
jobs_df = jobs_df.iloc[: , 1:]
print(jobs_df.shape)
google_df = google_df.iloc[: , 1:]
print(google_df.shape)



(12, 5)
(7, 5)


In [36]:
frames = [jobs_df, google_df]
jobs_df = pd.concat(frames)
jobs_df.shape

(19, 5)

In [37]:
jobs_df.head()

,company,description,salary,stacks,title
0,Accedo.tv,Accedo is looking for a talented Android engin...,"['4,125', '5,000']","['TDD', 'HTML', 'React Native', 'Android', 'Ko...",Software Engineer (Android)
1,Accedo.tv,Accedo is looking for a talented Android engin...,"['4,125', '5,000']","['TDD', 'HTML', 'React Native', 'Android', 'Ko...",Software Engineer (Android)
2,Accedo.tv,Accedo is looking for a talented Android engin...,"['4,125', '5,000']","['TDD', 'HTML', 'React Native', 'Android', 'Ko...",Software Engineer (Android)
3,GXS Bank,About The Team\n\nWhen people talk about Digib...,"['5,500', '8,250']","['TDD', 'CI', 'RxSwift', 'MVVM', 'iOS', 'Swift']","Software Engineer, Mobile (IOS)"
4,TikTok,"Why Join Us\nAt TikTok, our people are humble,...","['6,787', '13,620']","['Java', 'Android', 'Kotlin']",Android Software Engineer - TikTok Live


In [38]:
def str_to_list(string):
    return ast.literal_eval(string)

jobs_df['stacks'] = jobs_df['stacks'].apply(str_to_list)


In [39]:
jobs_df

,company,description,salary,stacks,title
0,Accedo.tv,Accedo is looking for a talented Android engin...,"['4,125', '5,000']","[TDD, HTML, React Native, Android, Kotlin, Jav...",Software Engineer (Android)
1,Accedo.tv,Accedo is looking for a talented Android engin...,"['4,125', '5,000']","[TDD, HTML, React Native, Android, Kotlin, Jav...",Software Engineer (Android)
2,Accedo.tv,Accedo is looking for a talented Android engin...,"['4,125', '5,000']","[TDD, HTML, React Native, Android, Kotlin, Jav...",Software Engineer (Android)
3,GXS Bank,About The Team\n\nWhen people talk about Digib...,"['5,500', '8,250']","[TDD, CI, RxSwift, MVVM, iOS, Swift]","Software Engineer, Mobile (IOS)"
4,TikTok,"Why Join Us\nAt TikTok, our people are humble,...","['6,787', '13,620']","[Java, Android, Kotlin]",Android Software Engineer - TikTok Live
5,LiquidX Studio,The Challenges\n\nYou will be involved in diff...,0,"[C++, Java, C, C#]",Game Developer (Remote)
6,TikTok,"Why Join Us\nAt TikTok, our people are humble,...","['6,786', '13,477']","[Graph, C++, Go, Linux, Python]",Algorithm Engineer (Platform Governance) - 202...
7,ST Engineering,"Key Responsibilities:\n\n• Perform design, dev...",0,"[C++, Java, Android, C#]",Android Developer (Entry level candidates welc...
8,Singlife with Aviva,The mobile developer is a software engineer wh...,"['4,000', '6,000']","[Strategy, AWS, Figma, Google Cloud, Firebase,...",Mobile Application Developer
9,Central Provident Fund (CPF) Board,What You Will Be Working On\n\nYou will use a ...,0,[],Associate Cybersecurity Consultant (Fresh Dipl...


In [40]:
records = []

for index, row in jobs_df.iterrows():
    records.append([stack for stack in row['stacks']])

print(len(records))

print(records[0:10])

19
[['TDD', 'HTML', 'React Native', 'Android', 'Kotlin', 'JavaScript'], ['TDD', 'HTML', 'React Native', 'Android', 'Kotlin', 'JavaScript'], ['TDD', 'HTML', 'React Native', 'Android', 'Kotlin', 'JavaScript'], ['TDD', 'CI', 'RxSwift', 'MVVM', 'iOS', 'Swift'], ['Java', 'Android', 'Kotlin'], ['C++', 'Java', 'C', 'C#'], ['Graph', 'C++', 'Go', 'Linux', 'Python'], ['C++', 'Java', 'Android', 'C#'], ['Strategy', 'AWS', 'Figma', 'Google Cloud', 'Firebase', 'OOP'], []]


In [41]:
itemsets, rules = apriori(records, min_support=0.01, min_confidence=0.5, max_length=15)

In [52]:
result_apr_df = pd.DataFrame(columns=['antecedent', 'consequent', 'basket_rule', 'support', 'confidence', 'lift'])
for rule in rules:
    antecedent = list(rule.lhs)
    consequent = list(rule.rhs)
    basket_rule = str(antecedent) + "->" + str(consequent)
    support = rule.support
    confidence = rule.confidence
    lift = rule.lift
    result_apr_df = result_apr_df.append({'antecedent': antecedent, 'consequent': consequent, 'basket_rule': basket_rule, 'support': support, 'confidence': confidence, 'lift': lift}, ignore_index=True)

result_apr_df = result_apr_df.sort_values(by=['confidence'], ascending=False)
result_apr_df.head()

/var/folders/t7/n8_8vs011h77724m2y7h54mc0000gn/T/ipykernel_52870/783505555.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_apr_df = result_apr_df.append({'antecedent': antecedent, 'consequent': consequent, 'basket_rule': basket_rule, 'support': support, 'confidence': confidence, 'lift': lift}, ignore_index=True)
/var/folders/t7/n8_8vs011h77724m2y7h54mc0000gn/T/ipykernel_52870/783505555.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_apr_df = result_apr_df.append({'antecedent': antecedent, 'consequent': consequent, 'basket_rule': basket_rule, 'support': support, 'confidence': confidence, 'lift': lift}, ignore_index=True)
/var/folders/t7/n8_8vs011h77724m2y7h54mc0000gn/T/ipykernel_52870/783505555.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a f

,antecedent,consequent,basket_rule,support,confidence,lift
0,[Figma],[AWS],['Figma']->['AWS'],0.052632,1.0,19.000000
3287,"[HTML, Kotlin, TDD]","[Android, React Native]","['HTML', 'Kotlin', 'TDD']->['Android', 'React ...",0.157895,1.0,6.333333
3294,"[Android, HTML, Kotlin]","[React Native, TDD]","['Android', 'HTML', 'Kotlin']->['React Native'...",0.157895,1.0,6.333333
3293,"[Android, HTML, React Native]","[Kotlin, TDD]","['Android', 'HTML', 'React Native']->['Kotlin'...",0.157895,1.0,6.333333
3292,"[Android, HTML, TDD]","[Kotlin, React Native]","['Android', 'HTML', 'TDD']->['Kotlin', 'React ...",0.157895,1.0,6.333333


In [53]:
result_apr_df = result_apr_df.rename_axis('index')

In [54]:
# result_apr_df.to_csv('results.csv')
result_apr_df.iloc[:100, :].to_csv('results.csv')

Load CSV files into MongoDB

In [55]:
import csv
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://35.171.48.20:27017')
db = client['IS459']
collection = db['analysis_results']

# Open CSV file and read rows into a list
with open('results.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    rows = list(reader)

# Define the filter to check if the document already exists
for i in range(100):
    row = rows[i]
# for row in rows:
    filter = {'antecedent': row['antecedent'], 'consequent': row['consequent']}
    
    # Define the update operation to set the document if it doesn't exist
    update = {'$setOnInsert': row}
    
    # Execute the update operation with upsert=True to insert the document if it doesn't exist
    result = collection.update_one(filter, update, upsert=True)
    
    if result.upserted_id is not None:
        print(f"Document for antecedent: {row['antecedent']} and consequent: {row['consequent']} inserted.")
    else:
        print(f"Document for antecedent: {row['antecedent']} and consequent: {row['consequent']} already exists.")


Document for antecedent: ['Figma'] and consequent: ['AWS'] inserted.
Document for antecedent: ['HTML', 'Kotlin', 'TDD'] and consequent: ['Android', 'React Native'] inserted.
Document for antecedent: ['Android', 'HTML', 'Kotlin'] and consequent: ['React Native', 'TDD'] inserted.
Document for antecedent: ['Android', 'HTML', 'React Native'] and consequent: ['Kotlin', 'TDD'] inserted.
Document for antecedent: ['Android', 'HTML', 'TDD'] and consequent: ['Kotlin', 'React Native'] inserted.
Document for antecedent: ['Android', 'Kotlin', 'React Native'] and consequent: ['HTML', 'TDD'] inserted.
Document for antecedent: ['Android', 'Kotlin', 'TDD'] and consequent: ['HTML', 'React Native'] inserted.
Document for antecedent: ['Android', 'React Native', 'TDD'] and consequent: ['HTML', 'Kotlin'] inserted.
Document for antecedent: ['HTML', 'Kotlin', 'React Native'] and consequent: ['Android', 'TDD'] inserted.
Document for antecedent: ['HTML', 'React Native', 'TDD'] and consequent: ['Android', 'Kotli

In [29]:
jobs_df.head()

,Unnamed: 0,company,title,stacks,salary,description
0,0,Accedo.tv,Software Engineer (Android),"[TDD, HTML, React Native, Android, Kotlin, Jav...","['4,125', '5,000']",Accedo is looking for a talented Android engin...
1,1,Accedo.tv,Software Engineer (Android),"[TDD, HTML, React Native, Android, Kotlin, Jav...","['4,125', '5,000']",Accedo is looking for a talented Android engin...
2,2,Accedo.tv,Software Engineer (Android),"[TDD, HTML, React Native, Android, Kotlin, Jav...","['4,125', '5,000']",Accedo is looking for a talented Android engin...
3,3,GXS Bank,"Software Engineer, Mobile (IOS)","[TDD, CI, RxSwift, MVVM, iOS, Swift]","['5,500', '8,250']",About The Team\n\nWhen people talk about Digib...
4,4,TikTok,Android Software Engineer - TikTok Live,"[Java, Android, Kotlin]","['6,787', '13,620']","Why Join Us\nAt TikTok, our people are humble,..."


In [50]:
stacks_df = pd.DataFrame(columns=['stack', 'count'])
stacks_map = {}

for row in jobs_df['stacks']:
    for item in row:
        if item not in stacks_map:
            stacks_map[item] = 1
        else:
            stacks_map[item] += 1
for stack, count in stacks_map.items():
    stacks_df = stacks_df.append({'stack': stack, 'count': count}, ignore_index=True)

stacks_df

C:\Users\shawn\AppData\Local\Temp\ipykernel_15476\3616466797.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stacks_df = stacks_df.append({'stack': stack, 'count': count}, ignore_index=True)
C:\Users\shawn\AppData\Local\Temp\ipykernel_15476\3616466797.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stacks_df = stacks_df.append({'stack': stack, 'count': count}, ignore_index=True)
C:\Users\shawn\AppData\Local\Temp\ipykernel_15476\3616466797.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stacks_df = stacks_df.append({'stack': stack, 'count': count}, ignore_index=True)
C:\Users\shawn\AppData\Local\Temp\ipykernel_15476\3616466797.py:11: FutureWarning: The frame.append method is deprecated and will be removed from

,stack,count
0,TDD,45
1,HTML,19
2,React Native,26
3,Android,27
4,Kotlin,13
...,...,...
254,WordPress,1
255,WebSphere,1
256,Pentaho,1
257,Data Mart,1


In [51]:
collection_stacks = db['stacks']

stack_records = stacks_df.to_dict('records')
collection_stacks.insert_many(stack_records)